In [10]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level


# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        self.sma_short_5=bt.indicators.SimpleMovingAverage(self.stock_px, period=5)
        self.sma_long_30=bt.indicators.SimpleMovingAverage(self.stock_px, period=30)
        self.sma_short_50 = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
        self.sma_long_200 = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
        self.ema_short=bt.indicators.ExponentialMovingAverage(self.stock_px,period=12)
        self.ema_long=bt.indicators.ExponentialMovingAverage(self.stock_px,period=26)
        self.macd=bt.indicators.MACD(self.stock_px)
        self.dema_short=bt.indicators.DoubleExponentialMovingAverage(self.stock_px,period=21)
        self.dema_long=bt.indicators.DoubleExponentialMovingAverage(self.stock_px,period=55)
        self.hma_30=bt.indicators.HullMA(self.stock_px,period=30)
        #self.psar=bt.indicators.PSAR(self.stock_px,period=2)
        #self.adx=bt.indicators.ADX(self.stock_px,period=14)
        
        #self.dsma = self.sma_short - self.sma_long
        #self.mom = bt.Cmp(self.sma_short, self.sma_long)
        #self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        #self.rsi_oversold = RSI_Oversold(self.stock_px)
        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30

        #self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        #self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        
        
#     def next(self):
#         # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
#         size = self.position.size
        
#         # Check if we are currently in a trade
#         if(not size):
#             #If not has our indicator triggered?
#             if self.rsi_oversold[0] > 0:
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#         else:
#             self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
#             if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
                

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/AAPL.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='APPL_trend_indicators.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
